# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dlc import analyze_new
from preprocess import fix_point, remove_cols, df2hdf, gen_anipose_files
import utils

c:\ProgramData\Anaconda3\envs\DEEPLABCUT\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
c:\ProgramData\Anaconda3\envs\DEEPLABCUT\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
c:\ProgramData\Anaconda3\envs\DEEPLABCUT\lib\site-packages\deeplabcut\__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


### STEP 0: Configuration

- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [3]:
# Filepath to the geontype folder containing videos
videos = r"C:\Users\bidayelab\Documents\SummerIntern\RawData\StochasticActivation"
videos = Path(videos)

p_networks = Path(r'./common_files/dlc_networks.yml')

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new(videos, p_networks)

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [4]:
# TODO: will probably handle all 3 below from a single directory
error = False

# find all the CSVs that DLC generated
parent_dir = ""
for p_csv in utils.get_csvs(videos):
    parent_dir = p_csv.parent.parent.parent

    # Pick current generated, filtered CSV 
    
    # TODO: also check for cam name and model name
    csv_name = p_csv.stem
    if "filtered" not in csv_name:
        continue

    print(f"[INFO] Processing {p_csv.name}")
    csv_df = utils.load_csv_as_df(p_csv)

    # Fix points
    print("[INFO] Running `Fix points` preprocessing...")
    col_names = ['TaG', 'Notum', 'WH']
    n = -1 # Values will be replaced with the nth entry. To replace with the mean, use n=0
    for name in col_names:
        print(f"[INFO] Matching string {name}")
        csv_df = fix_point(csv_df, name, n)

    # Remove cols 
    print("[INFO] Running `Remove cols` preprocessing...")
    camName = p_csv.name[0]
    start = ''
    end = ''
    if camName == 'B':
        print("[INFO] camName `B`, removing cols starting with `L-`")
        start = 'L-' # Remove col if start of name matches string
    if camName == 'E':
        print("[INFO] camName `E`, removing cols starting with `R-`")
        start = 'R-' # Remove col if start of name matches string
    csv_df = remove_cols(csv_df, start, end)

    # Rename and convert to .h5 (saves to original directory)
    print("[INFO] Converting CSV to HDF...")
    root = Path(r'C:\Users\bidayelab\Documents\SummerIntern\RawData')
    print(f"[INFO] Using root path {root}")
    if df2hdf(csv_df, p_csv, root): # error occured
        error = True


# Generate anipose file structure
p_calibration_target = Path(r"./common_files/calibration_target.yml") # calibration target config file 
p_calibration_timeline = Path(r"./common_files/calib_timeline.yml") # calibration timeline config file
calibration_type = utils.get_calibration_type(p_calibration_target, parent_dir)
if calibration_type == 'fly':
    p_anipose_config = Path(r"./common_files/config_fly.toml") # anipose config file
elif calibration_type == 'board':
    p_anipose_config = Path(r"./common_files/config_board.toml") # anipose config file
else:
    print(f"[ERROR] Invalid calibration type or calibration type not specified in {p_calibration_target}")
    error = True

# check that all the files exist
if not p_calibration_target.exists():
    print("[ERROR] `calibration_target.yml` does not exist.")
    error = True
if not p_calibration_timeline.exists():
    print("[ERROR] `calib_timeline.yml` does not exist.")
    error = True

if not error:
    print("[INFO] Generating anipose files...")
    gen_anipose_files(parent_dir, p_networks, p_anipose_config, p_calibration_target, p_calibration_timeline)

    print('Finished preprocessing...')
else:
    print("Terminated due to error")


[INFO] Processing A-03292023203948-0000DLC_resnet101_camA_augmentedJan18shuffle1_500000_filtered.csv


c:\Users\bidayelab\Documents\SummerIntern\Unified Pipeline\unified_pipeline\utils.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24

[INFO] Running `Fix points` preprocessing...
[INFO] Matching string TaG
INFO value in TaG replaced with []
[INFO] Matching string Notum
INFO value in Notum replaced with [749.29818738 382.96283721   0.99999976 749.29367398 382.96472136
   0.99999986 749.20514295 382.92177249   0.99999963 749.28162754
 383.15254759   0.99999947 749.49860959 383.07010858   0.99999921]
[INFO] Matching string WH
INFO value in WH replaced with []
[INFO] Running `Remove cols` preprocessing...
[INFO] Converting CSV to HDF...
[INFO] Using root path \\mpfi.org\public\sb-lab\BallSystem_RawData
[ERROR] Incorrect root.
Your root path does not match with the parent directory provided, please make sure that you provided the correct root.         
The root should be the beginning of your parent directory path up to the folder containing raw data, e.g `\mpfi.org\public\sb-lab\BallSystem_RawData`

[INFO] Processing B-03292023203954-0000DLC_resnet101_3cam_BEHSep16shuffle1_500000_filtered.csv


c:\Users\bidayelab\Documents\SummerIntern\Unified Pipeline\unified_pipeline\utils.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24

[INFO] Running `Fix points` preprocessing...
[INFO] Matching string TaG
INFO value in TaG replaced with []
[INFO] Matching string Notum
INFO value in Notum replaced with [8.09600878e+02 6.59742100e+02 1.53483012e-03 8.53145902e+02
 6.23957933e+02 1.10206493e-03 8.87270861e+02 6.17763091e+02
 9.51501710e-04 8.91842914e+02 6.17478425e+02 8.59824377e-04
 8.92770287e+02 6.17954058e+02 7.96235220e-04]
[INFO] Matching string WH
INFO value in WH replaced with []
[INFO] Running `Remove cols` preprocessing...
[INFO] camName `B`, removing cols starting with `L-`
[INFO] removed 240 columns starting with L-
[INFO] Converting CSV to HDF...
[INFO] Using root path \\mpfi.org\public\sb-lab\BallSystem_RawData
[ERROR] Incorrect root.
Your root path does not match with the parent directory provided, please make sure that you provided the correct root.         
The root should be the beginning of your parent directory path up to the folder containing raw data, e.g `\mpfi.org\public\sb-lab\BallSystem_RawDa

...

c:\Users\bidayelab\Documents\SummerIntern\Unified Pipeline\unified_pipeline\utils.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24

[INFO] Running `Fix points` preprocessing...
[INFO] Matching string TaG
INFO value in TaG replaced with []
[INFO] Matching string Notum
INFO value in Notum replaced with [7.73218956e+02 6.11985749e+02 3.84118517e-03 7.80084501e+02
 6.21483172e+02 2.95060074e-03 7.82755673e+02 6.23867947e+02
 2.40852506e-03 7.92249182e+02 6.25147096e+02 2.10320955e-03
 7.92231257e+02 6.33263693e+02 1.90259046e-03]
[INFO] Matching string WH
INFO value in WH replaced with []
[INFO] Running `Remove cols` preprocessing...
[INFO] camName `E`, removing cols starting with `R-`
[INFO] removed 240 columns starting with R-
[INFO] Converting CSV to HDF...
[INFO] Using root path \\mpfi.org\public\sb-lab\BallSystem_RawData
[ERROR] Incorrect root.
Your root path does not match with the parent directory provided, please make sure that you provided the correct root.         
The root should be the beginning of your parent directory path up to the folder containing raw data, e.g `\mpfi.org\public\sb-lab\BallSystem_RawDa

c:\Users\bidayelab\Documents\SummerIntern\Unified Pipeline\unified_pipeline\utils.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24

[INFO] Running `Fix points` preprocessing...
[INFO] Matching string TaG
INFO value in TaG replaced with []
[INFO] Matching string Notum
INFO value in Notum replaced with [732.19444102 440.11494312   0.99999994 732.19138429 440.11665271
   0.99999998 732.21172276 440.07162981   0.99999997 732.12083322
 440.26083873   0.99999991 732.03335134 440.33479457   0.99999986]
[INFO] Matching string WH
INFO value in WH replaced with []
[INFO] Running `Remove cols` preprocessing...
[INFO] Converting CSV to HDF...
[INFO] Using root path \\mpfi.org\public\sb-lab\BallSystem_RawData
[ERROR] Incorrect root.
Your root path does not match with the parent directory provided, please make sure that you provided the correct root.         
The root should be the beginning of your parent directory path up to the folder containing raw data, e.g `\mpfi.org\public\sb-lab\BallSystem_RawData`

[INFO] Processing H-03292023204055-0000DLC_resnet101_3cam_BEHSep16shuffle1_500000_filtered.csv
[INFO] Running `Fix points` 

c:\Users\bidayelab\Documents\SummerIntern\Unified Pipeline\unified_pipeline\utils.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24